In [1]:
from dataset_factory import DatasetFactory
from peft import LoraConfig, PeftConfig, PeftModel, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import evaluate as eval_metrics
import torch
import json


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /home/girinman/anaconda3/envs/hai-ground/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /home/girinman/anaconda3/envs/hai-ground/lib/libcudart.so
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /home/girinman/anaconda3/envs/hai-ground/lib/python3.9/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


/home/girinman/anaconda3/envs/hai-ground/lib/python3.9/site-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: Found duplicate ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] files: {PosixPath('/home/girinman/anaconda3/envs/hai-ground/lib/libcudart.so'), PosixPath('/home/girinman/anaconda3/envs/hai-ground/lib/libcudart.so.11.0')}.. We'll flip a coin and try one of these, in order to fail forward.
Either way, this might cause trouble in the future:
If you get `CUDA error: invalid device function` errors, the above might be the cause and the solution is to make sure only one ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] in the paths that we search based on your env.
  warn(msg)


In [2]:
ckpt_dir = "/home/girinman/repos/restaurant_suggestion_with_llms/template_based_SFT/checkpoints/polyglot-ko-1.3b-lora-parse_cmd/2023-05-15_05:31:19"
running_config = json.load(open(ckpt_dir + "/running_config.json", "r"))

In [3]:
model_id = running_config['model_name_or_path']
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,#running_config['train_in_8bit'],
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
).eval()

In [4]:
# Load adapters from the Hub and generate some output texts:

peft_model_id = ckpt_dir
lora_config = LoraConfig.from_pretrained(peft_model_id)
model = get_peft_model(model, lora_config)


In [5]:
lora_state = torch.load(peft_model_id + '/adapter_model.bin')
for n, m in model.named_modules():
    if n+'.weight' in lora_state:
        print(n)
        m.load_state_dict({'weight' :lora_state[n+'.weight']})

base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.0.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.1.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.1.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.2.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.2.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.3.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.3.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.4.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.4.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.5.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.5.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.6.attention.query_key_value.lora_A
base_model.model.gpt_neox.layers.6.attention.query_key_value.lora_B
base_model.model.gpt_neox.layers.7.attention.que

In [6]:
# load dataset
df = DatasetFactory(tokenizer)

loaded_dataset = df.load_dataset(
    name=running_config['dataset_name'],
    max_len=running_config['max_len'],
    batch_size=4#running_config['per_device_eval_batch_size'],
)

data = loaded_dataset.dataset
suffix = loaded_dataset.suffix
max_label_len = loaded_dataset.max_label_len

Found cached dataset csv (/home/girinman/.cache/huggingface/datasets/csv/default-b08ba15b77f0c96d/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/1 [00:00<?, ?it/s]

Parameter 'function'=<function CMD.get_prompt_dataset.<locals>.generate_prompt at 0x7f8748271040> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/1818 [00:00<?, ? examples/s]

Map:   0%|          | 0/96 [00:00<?, ? examples/s]

In [7]:
test_loader = loaded_dataset.get_test_dataloader()
for batch in test_loader:
    first_batch = batch
    break

In [8]:
with torch.cuda.amp.autocast():
    output_tokens = model.generate(
        input_ids=first_batch['input_ids'].to(model.device),
        max_new_tokens=88,
        eos_token_id = tokenizer.eos_token_id,
        pad_token_id = tokenizer.pad_token_id,
    )

/home/girinman/anaconda3/envs/hai-ground/lib/python3.9/site-packages/bitsandbytes/autograd/_functions.py:318: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/home/girinman/anaconda3/envs/hai-ground/lib/python3.9/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:220: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at /opt/conda/conda-bld/pytorch_1670525539683/work/aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_scores = torch.where(causal_mask, attn_scores, mask_value)


In [13]:
generated = tokenizer.batch_decode(output_tokens, skip_special_tokens=True)
for txt in generated:
    print(txt)
    print()

명령: 강남역 주변에 괜찮은 한식집 있을까요?
시간 : 알 수 없음 알 수 없음지역 : 서울특별시 강남구 신사동 가로수길 알 수 없음
시간 : 알 수 없음
지역 : 알 수 없음
시간 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음

명령: 성신여대 근처에 혼술하기 좋은 맥주집 추천해주세요.
시간 : 알 수 없음
시간 : 알 수 없음
지역 : 알 수 없음
시간 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음
지역 : 알 수 없음
요구사항 : 알 수 없음

명령: 서울 근교로 드라이브 갈만한 곳을 찾고 있어요. 어디가 좋을까요?
시간 : 알 수 없음
지역 : 서울특별시 강서구청역
시간 : 알 수 없음
지역 : 서울 강서구청역
시간 : 알 수 없음
지역 : 서울 강서구청역
요구사항 : 알 수 없음
지역 : 서울 강서구청역
요구사항 : 알 수 없음

명령: 강남역에서 소개팅 할만한 곳 추천해주세요.
시간 : 알 수 없음
지역 : 강남역 근처 커피숍, 커피숍, 커피숍 등
시간 : 알 수 없음
지역 : 강남역
시간 : 알 수 없음
요구사항 : 커피숍, 커피숍



In [10]:
import evaluate as eval_metrics
rouge = eval_metrics.load('rouge')

def validation_step(model, tokenizer, batch, first):
    with torch.cuda.amp.autocast():
        generated_ids = model.generate(
            input_ids=batch['input_ids'].to(model.device),
            attention_mask=batch['attention_mask'].to(model.device),
            max_new_tokens=max_label_len,
            eos_token_id = tokenizer.eos_token_id,
            pad_token_id = tokenizer.pad_token_id,
        )
    generated_txt = []
    for i, g in enumerate(generated_ids):
        decoded_txt = tokenizer.decode(g.tolist(), skip_special_tokens=True).split(suffix)
        generated_txt.append(decoded_txt[-1].strip())
    
    labels = batch['decoded_labels']

    return {'generated': generated_txt, 'labels': labels}

def validation_epoch_end(outputs):
    generated_txt = []
    labels = []
    preds = []

    for i in outputs:
        generated_txt.extend(i['generated'])
        labels.extend(i['labels'])
    
    metrics = rouge.compute(predictions=generated_txt,
                            references=labels,
                            # tokenizer=lambda x: x.split(),
                            )
    
    return metrics

def evaluate(eval_dataloader):
    eval_results = []
    for i, batch in tqdm(enumerate(eval_dataloader), "Generating predictions", total=len(eval_dataloader)):
        eval_results.append(validation_step(model, tokenizer, batch, i == 0))
    return validation_epoch_end(eval_results)

In [11]:
print(f"***Evaluation begins***")
metrics = evaluate(test_loader)
print(f"***Evaluation results***")
for key, value in metrics.items():
    print(f"{key}: {value}")

***Evaluation begins***


Generating predictions: 100%|██████████| 205/205 [19:04<00:00,  5.58s/it]


***Evaluation results***
rouge1: 0.518373921879556
rouge2: 0.2775653146251903
rougeL: 0.4383280042643704
rougeLsum: 0.4382960341521835


In [13]:
val_loader = loaded_dataset.get_eval_dataloader()

print(f"***Evaluation begins***")
metrics = evaluate(val_loader)
print(f"***Evaluation results***")
for key, value in metrics.items():
    print(f"{key}: {value}")

***Evaluation begins***


Generating predictions: 100%|██████████| 205/205 [19:25<00:00,  5.68s/it]


***Evaluation results***
rouge1: 0.5230088067234611
rouge2: 0.2892967975202029
rougeL: 0.44118840842498985
rougeLsum: 0.4407998007187565
